In [1]:
import yaml
from myclient import MyClient
import json
from rich import print
import requests
import base64
myclient=MyClient(logging=False)

In [2]:
with open("/home/mkhan/Codes/onshape-cad-parser/creds.json","r") as f:
    data=json.load(f)

data=list(data.values())[0]

In [3]:
data_id="00000029"
link="https://cad.onshape.com/api/partstudios/d/ad34a3f60c4a4caa99646600/w/90b1c0593d914ac7bdde17a3/e/f5cef14c36ad4428a6af59f0"

v_list = link.split("/")
did, wid, eid = v_list[-5], v_list[-3], v_list[-1]

url = f"https://cad.onshape.com/api/documents/{did}/workspaces/{wid}/elements/{eid}/features"
params = {}
credentials = (data["access_key"],data["secret_key"])

# Define the header for the request 
headers = {'Accept': 'application/json;charset=UTF-8;qs=0.09',
           'Content-Type': 'application/json'}

# # Putting everything together to make the API request 
# response = requests.get(link, 
#                         params=params, 
#                         auth=credentials, 
#                         headers=headers)

# res=response.json()

In [5]:
import re,math
def deg2rad(expr):
    """Convert value expression to radians unit"""
    # Use regular expression to find all numeric values in the expression string
    numeric_values = re.findall(r'[+-]?\d+(?:\.\d+)?', expr)

    # Convert each numeric value to radians and sum them up
    radians = sum(math.radians(float(value)) for value in numeric_values)
    return radians

In [7]:
deg2rad("30.330*deg")

0.5293583621298801

In [5]:
response=myclient.get_entity_by_id(did,wid,eid,['JLH'],'FACE')

In [9]:
myclient.get_entity_by_id(did,wid,eid,['JLH'],'FACE').json()

{'console': '',
 'notices': [{'type': 227,
   'typeName': 'BTNotice',
   'message': {'level': 'WARNING',
    'type': 'SEMANTIC',
    'message': 'Variable console not found.',
    'stackTrace': [{'type': 226,
      'typeName': 'BTLocationInfo',
      'message': {'document': 'f5cef14c36ad4428a6af59f0',
       'version': '28970fda968c205f58c8408a',
       'topLevel': '',
       'parseNodeId': 'P1urxboNfF6soAPD9',
       'character': 0,
       'line': 0,
       'column': 0,
       'endCharacter': 0,
       'endLine': 0,
       'endColumn': 0,
       'languageVersion': 0,
       'moduleIds': {'type': 0},
       'elementMicroversion': '28970fda968c205f58c8408a',
       'nodeId': 'iV+a0cIwbyqrt/cq'}}],
    'tryNode': {'type': 0},
    'parameterId': '',
    'nodeId': 'oIckOsqmy21lMaID'}},
  {'type': 227,
   'typeName': 'BTNotice',
   'message': {'level': 'ERROR',
    'type': 'SEMANTIC',
    'message': 'Variable console not found.',
    'stackTrace': [{'type': 226,
      'typeName': 'BTLocation

In [7]:
response.json()['result']

{'type': 0}

In [51]:
message=response.json()['result']['message']['value']

KeyError: 'message'

In [39]:
print(message[0])

{
    'type': 2062,
    'typeName': 'BTFSValueMap',
    'message': {
        'value': [
            {
                'type': 2077,
                'typeName': 'BTFSValueMapEntry',
                'message': {
                    'key': {
                        'type': 1422,
                        'typeName': 'BTFSValueString',
                        'message': {'value': 'direction', 'typeTag': ''}
                    },
                    'value': {
                        'type': 1499,
                        'typeName': 'BTFSValueArray',
                        'message': {
                            'value': [
                                {
                                    'type': 772,
                                    'typeName': 'BTFSValueNumber',
                                    'message': {'value': 1.0112740684418475e-15, 'typeTag': ''}
                                },
                                {
                                    'type': 772,
                                    'typeName': 'BTFSValueNumber',
                                    'message': {'value': 0.0, 'typeTag': ''}
                                },
                                {
                                    'type': 772,
                                    'typeName': 'BTFSValueNumber',
                                    'message': {'value': 1.0, 'typeTag': ''}
                                }
                            ],
                            'typeTag': 'Vector'
                        }
                    }
                }
            },
            {
                'type': 2077,
                'typeName': 'BTFSValueMapEntry',
                'message': {
                    'key': {
                        'type': 1422,
                        'typeName': 'BTFSValueString',
                        'message': {'value': 'origin', 'typeTag': ''}
                    },
                    'value': {
                        'type': 1499,
                        'typeName': 'BTFSValueArray',
                        'message': {
                            'value': [
                                {
                                    'type': 1817,
                                    'typeName': 'BTFSValueWithUnits',
                                    'message': {
                                        'value': 0.0,
                                        'unitToPower': [{'key': 'METER', 'value': 1}],
                                        'typeTag': ''
                                    }
                                },
                                {
                                    'type': 1817,
                                    'typeName': 'BTFSValueWithUnits',
                                    'message': {
                                        'value': 0.0,
                                        'unitToPower': [{'key': 'METER', 'value': 1}],
                                        'typeTag': ''
                                    }
                                },
                                {
                                    'type': 1817,
                                    'typeName': 'BTFSValueWithUnits',
                                    'message': {
                                        'value': -0.001115364721044898,
                                        'unitToPower': [{'key': 'METER', 'value': 1}],
                                        'typeTag': ''
                                    }
                                }
                            ],
                            'typeTag': 'Vector'
                        }
                    }
                }
            }
        ],
        'typeTag': 'Line'
    }
}

In [49]:
axis_dict = {}
for item in message:
    for entry in item["message"]["value"]:
        key = entry['message']["key"]["message"]["value"]
        if key in ['direction','origin']:
            value_list = entry['message']["value"]["message"]["value"]
            val=[]
            for vl in value_list:
                if key=="origin" and vl['message']['unitToPower'][0]['key'].lower()=="meter":
                    val.append(myclient.expr2meter(did,eid,wid,vl['message']['value']))
                else:
                    val.append(vl['message']['value'])

        axis_dict[key] = val

KeyError: 'result'

In [47]:
axis_dict['origin']

[0.0, 0.0, -0.001115364721044898]

In [26]:
import requests
import json

data_id = "00000029"
link = "https://cad.onshape.com/documents/ad34a3f60c4a4caa99646600/w/90b1c0593d914ac7bdde17a3/e/f5cef14c36ad4428a6af59f0"

v_list = link.split("/")
did, wid, eid = v_list[-5], v_list[-3], v_list[-1]

url = f"https://cad.onshape.com/api/documents/{did}/workspaces/{wid}/elements/{eid}/features"

credentials = f'{data["access_key"]}:{data["secret_key"]}'
base64_encoded_credentials = base64.b64encode(credentials.encode()).decode()

headers = {
    "Authorization": f"Basic {base64_encoded_credentials}"
}
response = requests.get(url, headers=headers)

if response.status_code == 200:
    features = json.loads(response.content)
    for feature in features:
        operations = feature["operations"]
        for operation in operations:
            print(operation["name"])
else:
    print(response.status_code)


404